# neural analysis
Assesses data from neuropixels recordings

In [1]:
# imports 
import numpy as np 
import pandas as pd 
import statsmodels.api as sm
import h5py
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", message="Mean of empty slice", category=RuntimeWarning)

In [2]:
# functions
def get_labelled_posteriors(indata, labels):

    '''
    INPUTS:
    indata = posterior probabilites from a classifier with the shape
            n_trials x n_timesteps x n_classes
        
    labels = 1d array with len(n_trials) - these labels ought
            to correspond to class numbers (layers in indata)

    OUTPUT:
        labelled_posteriors = posterior probabilities associated with the
        classes in the labels input for each timestep and trial
    '''

    n_trials, n_times, n_classes = indata.shape
    class_lbls = np.unique(labels)
    class_lbls = class_lbls[~np.isnan(class_lbls)]

    # initialize output
    labelled_posteriors = np.zeros(shape = (n_trials, n_times))

    for ix, lbl in enumerate(class_lbls):
        
        # find trials where this label was chosen
        labelled_posteriors[labels == lbl,:] = indata[labels == lbl,:,int(ix)]
        
    return labelled_posteriors


def pull_balanced_train_set(trials2balance, params2balance):
    '''
    INPUTS:
    trials2balance   - ***logical array*** of the trials you want to balance
    params2balance   - ***list*** where each element is a vector of categorical
                        parameters to balance (e.g. choice value and side)
                        each element of params2balance must have the same
                        number of elements as trials2balance
    OUTPUTS:
    train_ix         - trial indices of a fully balanced training set
    leftover_ix      - trial indices of trials not included in train_ix
    '''

    # Find the indices where trials are selected to balance
    balance_indices = np.where(trials2balance)[0]

    # Create an array of parameters to balance
    params_array = np.array(params2balance).T

    # Find unique combinations and their counts
    p_combos, p_counts = np.unique(params_array[balance_indices], axis=0, return_counts=True)

    # Determine the minimum count for a balanced set
    n_to_keep = np.min(p_counts)

    # Initialize arrays to mark selected and leftover trials
    train_ix = np.zeros(len(trials2balance), dtype=bool)
    leftover_ix = np.zeros(len(trials2balance), dtype=bool)

    # Select a balanced number of trials for each unique parameter combination
    for combo in p_combos:
        # Find indices of trials corresponding to the current combination
        combo_indices = np.where((params_array == combo).all(axis=1) & trials2balance)[0]

        # Shuffle the indices
        np.random.shuffle(combo_indices)

        # Select n_to_keep trials and mark them as part of the training set
        train_ix[combo_indices[:n_to_keep]] = True

        # Mark the remaining trials as leftovers
        leftover_ix[combo_indices[n_to_keep:]] = True

    return train_ix, leftover_ix


def random_prop_of_array(inarray, proportion):
    '''
    INPUTS
    inarray = logical/boolean array of indices to potentially use later
    proportion = how much of inarray should randomly be selected

    OUTPUT
    out_array = logical/boolean that's set as 'true' for a proportion of the 
                initial 'true' values in inarray
    '''

    out_array = np.zeros(shape = (len(inarray), ))

    # find where inarray is true and shuffle those indices
    shuffled_ixs = np.random.permutation(np.asarray(np.where(inarray)).flatten())

    # keep only a proportion of that array
    kept_ix = shuffled_ixs[0: round(len(shuffled_ixs)*proportion)]

    # fill in the kept indices
    out_array[kept_ix] = 1

    # make this a logical/boolean
    out_array = out_array > 0

    return out_array


def pull_from_h5(file_path, data_to_extract):
    try:
        with h5py.File(file_path, 'r') as file:
            # Check if the data_to_extract exists in the HDF5 file
            if data_to_extract in file:
                data = file[data_to_extract][...]  # Extract the data
                return data
            else:
                print(f"'{data_to_extract}' not found in the file.")
                return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def list_hdf5_data(file_path):
    try:
        with h5py.File(file_path, 'r') as file:
            print(f"Datasets in '{file_path}':")
            for dataset in file:
                print(dataset)
    except Exception as e:
        print(f"An error occurred: {e}")


def get_ch_and_unch_vals(bhv):
    """
    Extracts chosen (ch_val) and unchosen (unch_val) values associated with each trial.

    Parameters:
    - bhv (DataFrame): DataFrame behavioral data.

    Returns:
    - ch_val (ndarray): Array of chosen values for each trial.
    - unch_val (ndarray): Array of unchosen values for each trial. 
                          - places 0s for unchosen values on forced choice trials
    """
    ch_val = np.zeros(shape=(len(bhv, )))
    unch_val = np.zeros(shape=(len(bhv, )))

    bhv['r_val'] = bhv['r_val'].fillna(0)
    bhv['l_val'] = bhv['l_val'].fillna(0)

    ch_left = bhv['side'] == -1
    ch_right = bhv['side'] == 1

    ch_val[ch_left] = bhv['l_val'].loc[ch_left].astype(int)
    ch_val[ch_right] = bhv['r_val'].loc[ch_right].astype(int)

    unch_val[ch_left] = bhv['r_val'].loc[ch_left].astype(int)
    unch_val[ch_right] = bhv['l_val'].loc[ch_right].astype(int)

    return ch_val, unch_val


def get_ch_and_unch_pps(in_pp, bhv, ch_val, unch_val):
    """Gets the posteriors associated with the chosen and unchosen classes

    Args:
        in_pp (ndarray): array of posteriors (n_trials x n_times x n_classes)
        bhv (dataframe): details of each trial
        ch_val (ndarray): vector indicating the class that is ultimately chosen
        unch_val (ndarray): vector indicating the class that was ultimately not chosen

    Returns:
        ch_pp (ndarray): vector of the postior at each point in time for each trial's chosen option
        unch_pp (ndarray): vector of the postior at each point in time for each trial's unchosen option
    """

    # select the chosen and unchosen values 
    n_trials, n_times, n_classes = np.shape(in_pp)
    ch_pp = np.zeros(shape=(n_trials, n_times))
    unch_pp = np.zeros(shape=(n_trials, n_times))

    # loop over each trial
    for t in range(n_trials):
        
        # get the chosen and unchosen PPs
        ch_pp[t, :] = in_pp[t, :, int(ch_val[t]-1)]
        unch_pp[t, :] = in_pp[t, :, int(unch_val[t]-1)]
        
    # set the forced choice unchosen pps to nans, since there was only 1 option
    unch_pp[bhv['forced'] == 1, :] = np.nan
    
    return ch_pp, unch_pp


def get_alt_ch_and_unch_pps(in_pp, bhv, s_ch_val, s_unch_val):
    """Gets the posteriors associated with the chosen and unchosen classes

    Args:
        in_pp (ndarray): array of posteriors (n_trials x n_times x n_classes)
        bhv (dataframe): details of each trial
        s_ch_val (ndarray): vector indicating the class that is ultimately chosen
        s_unch_val (ndarray): vector indicating the class that was ultimately not chosen

    Returns:
        alt_ch_pp (ndarray): vector of the postior at each point in time for the alternative value in the other state
        alt_unch_pp (ndarray): vector of the postior at each point in time for the alternative value in the other state
    """

    # select the chosen and unchosen values 
    n_trials, n_times, n_classes = np.shape(in_pp)
    alt_ch_pp = np.zeros(shape=(n_trials, n_times))
    alt_unch_pp = np.zeros(shape=(n_trials, n_times))

    alt_ch_val = np.zeros_like(s_ch_val)
    alt_unch_val = np.zeros_like(s_unch_val)
    
    alt_ch_val[bhv['state'] == 1] = 8 - s_ch_val[bhv['state'] == 1] + 1
    alt_ch_val[bhv['state'] == 2] = 8 - s_ch_val[bhv['state'] == 2] + 1

    alt_unch_val[bhv['state'] == 1] = 8 - s_unch_val[bhv['state'] == 1] + 1
    alt_unch_val[bhv['state'] == 2] = 8 - s_unch_val[bhv['state'] == 2] + 1

    for t in range(n_trials):
        
        alt_ch_pp[t, :] = in_pp[t, :, int(alt_ch_val[t]-1)]
        alt_unch_pp[t, :] = in_pp[t, :, int(alt_unch_val[t]-1)]

    # set the alternative values to nans for state 3, since there were no alternatives
    alt_ch_pp[bhv['state'] == 3] = np.nan
    alt_unch_pp[bhv['state'] == 3] = np.nan

    return alt_ch_pp, alt_unch_pp

def find_candidate_states(indata, n_classes, temporal_thresh, mag_thresh):
    """Finds periods where decoded posteriors are twice their noise level.

    Args:
        indata (ndarray): 2d array of posterior probabilities associated with some decoder output.
        n_classes (int): How many classes were used in the decoder?
        temporal_thresh (int): Number of contiguous samples that must be above a threshold to be a real state (typically 2).
        mag_thresh (flat): how many times the noise level must a state be? (e.g. 2 = twice the noise level)

    Returns:
        state_details (ndarray): 2d array where each row details when a state occurred [trial_num, time_in_trial, state_length].
        state_array (ndarray): 2d array the same size as indata. It contains 1 in all locations where there were states and 0s everywhere else.
    """
    state_details = np.array([])
    state_array = np.zeros_like(indata)
    
    state_magnitude_thresh = (1 / n_classes) * mag_thresh

    for t in range(indata.shape[0]):
        state_len, state_pos, state_type = find_1dsequences(indata[t, :] > state_magnitude_thresh)
        state_len = state_len[state_type == True]
        state_pos = state_pos[state_type == True]

        for i in range(len(state_len)):
            state_details = np.concatenate((state_details, np.array([t, state_pos[i], state_len[i]])))

    state_details = state_details.reshape(-1, 3)
    state_details = state_details[state_details[:, 2] > temporal_thresh, :]

    # Update state_array using state_details information
    for j in range(len(state_details)):
        state_trial, state_start, state_len = state_details[j].astype(int)
        state_array[state_trial, state_start:(state_start + state_len)] = 1

    return state_details, state_array

def moving_average(x, w, axis=0):
    '''
    Moving average function that operates along specified dimensions of a NumPy array.

    Parameters:
    - x (numpy.ndarray): Input array.
    - w (int): Size of the window to convolve the array with (i.e., smoothness factor).
    - axis (int): Axis along which to perform the moving average (default is 0).

    Returns:
    - numpy.ndarray: Smoothed array along the specified axis with the same size as the input array.
    '''
    x = np.asarray(x)  # Ensure input is a NumPy array
    if np.isnan(x).any():
        x = np.nan_to_num(x)  # Replace NaN values with zeros

    if axis < 0:
        axis += x.ndim  # Adjust negative axis value

    kernel = np.ones(w) / w  # Create kernel for moving average

    # Pad the array before applying convolution
    pad_width = [(0, 0)] * x.ndim  # Initialize padding for each axis
    pad_width[axis] = (w - 1, 0)  # Pad along the specified axis (left side)
    x_padded = np.pad(x, pad_width, mode='constant', constant_values=0)

    # Apply 1D convolution along the specified axis on the padded array
    return np.apply_along_axis(lambda m: np.convolve(m, kernel, mode='valid'), axis, x_padded)

def find_1dsequences(inarray):
        ''' 
        run length encoding. Partial credit to R rle function. 
        Multi datatype arrays catered for including non Numpy
        returns: tuple (runlengths, startpositions, values) 
        '''
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]                 # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)     # must include last element 
            lens = np.diff(np.append(-1, i))      # run lengths
            pos = np.cumsum(np.append(0, lens))[:-1] # positions
            return(lens, pos, ia[i])
  
        
def shuffle_along_axis(arr, axis):
    return np.apply_along_axis(np.random.permutation, axis, arr)


In [3]:
file_path = 'C:/Users/thome/Documents/PYTHON/Self-Control/raw_data/K20240707_Rec06.h5'
save_dir = 'C:/Users/thome/Documents/PYTHON/Self-Control/lr_decoder_output/' 
save_data = True

In [4]:
# access the data for this session
firing_rates = np.concatenate([pull_from_h5(file_path, 'CdN_zFR'), 
                               pull_from_h5(file_path, 'OFC_zFR')], axis=2)

u_names = np.concatenate([pull_from_h5(file_path, 'CdN_u_names'), 
                          pull_from_h5(file_path, 'OFC_u_names')], axis=0)

n_OFC = pull_from_h5(file_path, 'OFC_zFR').shape[2]
n_CdN = pull_from_h5(file_path, 'CdN_zFR').shape[2]
brain_areas = np.concatenate([np.zeros(shape=n_CdN, ), np.ones(shape=n_OFC, )]).astype(int)

ts = pull_from_h5(file_path, 'ts')
bhv = pd.read_hdf(file_path, key='bhv')

if len(bhv) > len(firing_rates):
    bhv = bhv.loc[0 :len(firing_rates)-1]

firing_rates = np.nan_to_num(firing_rates, nan=0)

In [5]:
# pull the relevant labels
ch_val, unch_val = get_ch_and_unch_vals(bhv)

# set up a state-independent value decoder
n_boots = 1000
n_trials = len(bhv)
n_vals = len(np.unique(ch_val))

# run a state x value classifier

# get the labels associated with each unique state-value pair
ch_val, unch_val = get_ch_and_unch_vals(bhv)
s_ch_val = ch_val.copy()
s_unch_val = unch_val.copy()

s_ch_val[bhv['state'] == 2] = s_ch_val[bhv['state'] == 2] + 4
s_ch_val[bhv['state'] == 3] = s_ch_val[bhv['state'] == 3] + 8
s_unch_val[bhv['state'] == 2] = s_unch_val[bhv['state'] == 2] + 4
s_unch_val[bhv['state'] == 3] = s_unch_val[bhv['state'] == 3] + 8

n_vals = len(np.unique(s_ch_val))

# initialize arrays to accumulate results into
OFC_pp = np.zeros(shape=(n_trials, len(ts), n_vals, n_boots), dtype=np.float32)
CdN_pp = np.zeros(shape=(n_trials, len(ts), n_vals, n_boots), dtype=np.float32)

OFC_acc = np.zeros(shape=(n_trials, len(ts), n_boots), dtype=np.float32)
CdN_acc = np.zeros(shape=(n_trials, len(ts), n_boots), dtype=np.float32)

# set those arrays to nans
OFC_pp[:] = np.nan
CdN_pp[:] = np.nan

OFC_acc[:] = np.nan
CdN_acc[:] = np.nan

ofc_ix = brain_areas == 1
cdn_ix = brain_areas == 0

In [6]:
# # loop over bootstraps
# for b in tqdm(range(n_boots)):

#     # partition the data into training and testing sets
#     train_trials = (bhv['forced'] == 1) | (bhv['n_sacc'] == 1)
#     # b_triasl2balance = random_prop_of_array(train_trials, .9)
#     # train_ix, leftover_ix = pull_balanced_train_set(b_triasl2balance, [ch_val, bhv['state'].values])
    
#     train_ix = random_prop_of_array(train_trials, .9)

#     train_labels = s_ch_val[train_ix]
#     train_fr = firing_rates[train_ix, :, :]
#     train_fr = np.nan_to_num(train_fr, nan=0)
    
#     # shuffle the training labels
#     s_train_labels = np.random.permutation(train_labels)
    
#     if len(np.unique(train_labels)) == len(np.unique(s_ch_val)):

#         # loop over time steps
#         for t in range(len(ts)-1):
            
#             # initialize classifiers
#             OFC_lda = LinearDiscriminantAnalysis()
#             CdN_lda = LinearDiscriminantAnalysis()

#             # train the classifiers
#             OFC_lda.fit(train_fr[:, t,  brain_areas == 1], train_labels)
#             CdN_lda.fit(train_fr[:, t,  brain_areas == 0], train_labels)

#             # test the classifiers 
#             OFC_pp[:, t, :, b] = OFC_lda.predict_proba(firing_rates[:, t, ofc_ix])
#             CdN_pp[:, t, :, b] = CdN_lda.predict_proba(firing_rates[:, t, cdn_ix])
            
#             OFC_acc[:, t, b] = OFC_lda.predict(firing_rates[:, t, ofc_ix]) == s_ch_val
#             CdN_acc[:, t, b] = CdN_lda.predict(firing_rates[:, t, cdn_ix]) == s_ch_val
            
#             # # shuffle the firing rates and test again
#             # s_OFC_pp[:, t, :, b] = OFC_lda.predict_proba(shuffle_along_axis(firing_rates[:, t, ofc_ix], axis=0))
#             # s_CdN_pp[:, t, :, b] = CdN_lda.predict_proba(shuffle_along_axis(firing_rates[:, t, cdn_ix], axis=0))

#         # set the training trials to nans 
#         OFC_pp[train_ix, :,:, b] = np.nan
#         CdN_pp[train_ix, :,:, b] = np.nan
        
#         # s_OFC_pp[train_ix, :,:, b] = np.nan
#         # s_CdN_pp[train_ix, :,:, b] = np.nan
        
#         OFC_acc[train_ix, :,b] = np.nan
#         CdN_acc[train_ix, :,b] = np.nan

# # average over the bootstraps for the real data
# OFC_pp_mean = np.nanmean(OFC_pp, axis=3)
# CdN_pp_mean = np.nanmean(CdN_pp, axis=3)

# OFC_acc_mean = np.nanmean(OFC_acc, axis=2)
# CdN_acc_mean = np.nanmean(CdN_acc, axis=2)

In [7]:
def perform_bootstrap(b):
    """
    Perform a single bootstrap iteration, including training and evaluating classifiers.
    """
    # Partition the data into training and testing sets
    train_trials = (bhv['forced'] == 1) | (bhv['n_sacc'] == 1)
    train_ix = random_prop_of_array(train_trials, .8)

    train_labels = s_ch_val[train_ix]
    train_fr = firing_rates[train_ix, :, :]
    train_fr = np.nan_to_num(train_fr, nan=0)

    # Shuffle the training labels for null distribution
    #s_train_labels = np.random.permutation(train_labels)

    # Check if all values are present
    if len(np.unique(train_labels)) == len(np.unique(s_ch_val)):
        
        # Initialize arrays to accumulate results
        OFC_pp_temp = np.full((n_trials, len(ts), n_vals), np.nan, dtype=np.float32)
        CdN_pp_temp = np.full((n_trials, len(ts), n_vals), np.nan, dtype=np.float32)
        
        OFC_acc_temp = np.full((n_trials, len(ts)), np.nan, dtype=np.float32)
        CdN_acc_temp = np.full((n_trials, len(ts)), np.nan, dtype=np.float32)
        
        # Loop over time steps
        for t in range(len(ts)-1):
            # # Initialize classifiers
            # OFC_lda = LinearDiscriminantAnalysis(priors=np.ones(n_vals) / n_vals)
            # CdN_lda = LinearDiscriminantAnalysis(priors=np.ones(n_vals) / n_vals)
            
            OFC_lda = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)
            CdN_lda = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)

            # Train the classifiers
            OFC_lda.fit(train_fr[:, t, brain_areas == 1], train_labels)
            CdN_lda.fit(train_fr[:, t, brain_areas == 0], train_labels)

            # Test the classifiers
            OFC_pp_temp[:, t, :] = OFC_lda.predict_proba(firing_rates[:, t, ofc_ix])
            CdN_pp_temp[:, t, :] = CdN_lda.predict_proba(firing_rates[:, t, cdn_ix])

            OFC_acc_temp[:, t] = OFC_lda.predict(firing_rates[:, t, ofc_ix]) == s_ch_val
            CdN_acc_temp[:, t] = CdN_lda.predict(firing_rates[:, t, cdn_ix]) == s_ch_val

        # Set training trials to NaNs
        OFC_pp_temp[train_ix, :, :] = np.nan
        CdN_pp_temp[train_ix, :, :] = np.nan

        OFC_acc_temp[train_ix, :] = np.nan
        CdN_acc_temp[train_ix, :] = np.nan

        return OFC_pp_temp, CdN_pp_temp, OFC_acc_temp, CdN_acc_temp
    else:
        # Return None if condition not met
        return None


In [8]:
# Run bootstraps in parallel
results = Parallel(n_jobs=6)(delayed(perform_bootstrap)(b) for b in tqdm(range(n_boots)))

# Aggregate results
for b, res in enumerate(results):
    if res is not None:
        OFC_pp[:, :, :, b], CdN_pp[:, :, :, b], OFC_acc[:, :, b], CdN_acc[:, :, b] = res
        
# Compute mean over bootstraps 
OFC_pp_mean = np.nanmean(OFC_pp, axis=3)
CdN_pp_mean = np.nanmean(CdN_pp, axis=3)

OFC_acc_mean = np.nanmean(OFC_acc, axis=2)
CdN_acc_mean = np.nanmean(CdN_acc, axis=2)

  4%|▎         | 18/500 [00:14<05:45,  1.39it/s]

In [ ]:
# get the chosen and unchosen posteriors
OFC_ch, OFC_unch = get_ch_and_unch_pps(OFC_pp_mean, bhv, s_ch_val, s_unch_val)
CdN_ch, CdN_unch = get_ch_and_unch_pps(CdN_pp_mean, bhv, s_ch_val, s_unch_val)

# get values associated with competing state
OFC_alt_ch, OFC_alt_unch = get_alt_ch_and_unch_pps(OFC_pp_mean, bhv, s_ch_val, s_unch_val)
CdN_alt_ch, CdN_alt_unch = get_alt_ch_and_unch_pps(CdN_pp_mean, bhv, s_ch_val, s_unch_val)

# get the alternative values associated with each option
ch_val, unch_val = get_ch_and_unch_vals(bhv)
ix = (bhv['n_sacc'] == 2) & (bhv['state'] == 2)

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.plot(ts, np.nanmean(OFC_acc_mean[ix, :], axis=0), color='tab:red', label='OFC')
ax1.plot(ts, np.nanmean(CdN_acc_mean[ix, :], axis=0), color='tab:blue', label='CdN')
ax1.set_xlabel('Time from Pics On (ms)')
ax1.set_ylabel('Accuracy')
ax1.legend()

ax2.plot(ts, np.nanmean(OFC_ch[ix, :], axis=0), color = 'tab:red', label='OFC_ch')
ax2.plot(ts, np.nanmean(OFC_alt_ch[ix, :], axis=0), color = 'tab:red', label='OFC_alt_ch', linestyle='--')
ax2.plot(ts, np.nanmean(OFC_unch[ix, :], axis=0), color = 'tab:red', linestyle=':', label='OFC_unch')
ax2.plot(ts, np.nanmean(OFC_alt_unch[ix, :], axis=0), color = 'tab:red', marker='o', label='OFC_alt_unch')
ax2.plot(ts, np.nanmean(CdN_ch[ix, :], axis=0), color = 'tab:blue', label='CdN_ch')
ax2.plot(ts, np.nanmean(CdN_alt_ch[ix, :], axis=0), color = 'tab:blue', label='CdN_alt_ch', linestyle='--')
ax2.plot(ts, np.nanmean(CdN_unch[ix, :], axis=0), color = 'tab:blue', linestyle=':', label='CdN_unch')
ax2.plot(ts, np.nanmean(CdN_alt_unch[ix, :], axis=0), color = 'tab:blue', marker='o', label='CdN_alt_unch')
ax2.set_xlabel('Time from Pics On (ms)')
ax2.set_ylabel('Posterior Probability')
ax2.legend()

In [ ]:
# save the data
save_name = save_dir + Path(file_path).stem + '_decoder.h5'

In [ ]:
if save_data:
    # Open an HDF5 file in write mode ('w' or 'w-' to create or truncate the file)
    with h5py.File(save_name, 'w') as file:
        # Create datasets within the HDF5 file and write data
        file.create_dataset('OFC_acc_mean', data=OFC_acc_mean)  
        file.create_dataset('OFC_ch', data=OFC_ch)  
        file.create_dataset('OFC_unch', data=OFC_unch)  
        file.create_dataset('OFC_alt_ch', data=OFC_alt_ch)  
        file.create_dataset('OFC_alt_unch', data=OFC_alt_unch)  
        file.create_dataset('OFC_pp', data=OFC_pp_mean)  
        
        file.create_dataset('CdN_acc_mean', data=CdN_acc_mean)  
        file.create_dataset('CdN_ch', data=CdN_ch)  
        file.create_dataset('CdN_unch', data=CdN_unch)  
        file.create_dataset('CdN_alt_ch', data=CdN_alt_ch)  
        file.create_dataset('CdN_alt_unch', data=CdN_alt_unch)  
        file.create_dataset('CdN_pp', data=CdN_pp_mean)

        file.create_dataset('ts', data= ts)
        
        # Save behavior to the HDF5 file
        bhv.to_hdf(save_name, key='bhv', mode='a')
    
